## Data Preparation
To integrate external data to enhance analysis by doing features engineering to enable deeper insights.

In [1]:
import pandas as pd

# Load CSV files into DataFrames
df = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
locations = pd.read_csv('ALL Prices 1990-2021 mar.csv')
malls = pd.read_csv('shopping_mall_coordinates.csv')
mrts = pd.read_csv('TrainStationLocation.csv')

### Data Cleaning

In [2]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [3]:
locations.head()

,month,year,town,town_dummy,flat_type,block,street_name,address,latitude,longitude,...,price_psm_yearly,Core CPI,price cpi_adj,price_psm cpi_adj,bala lease pct,price lease_adj implied,price_psm lease_adj implied,price cpi_lease_adj implied,price_psm cpi_lease_adj implied,year_gni
0,1990-01,1990,ANG MO KIO,2,4 ROOM,308,ANG MO KIO AVE 1,308 ANG MO KIO AVE 1 SINGAPORE,1.365485,103.844025,...,10.997442,61.59,139633.05730,1517.750623,92.9,88869.752422,965.975570,144292.502700,1568.396769,22901
1,1990-01,1990,ANG MO KIO,2,3 ROOM,308,ANG MO KIO AVE 1,308 ANG MO KIO AVE 1 SINGAPORE,1.365485,103.844025,...,8.464849,61.59,95794.77188,1168.228925,92.9,60968.783638,743.521752,98991.368143,1207.211806,22901
2,1990-01,1990,ANG MO KIO,2,3 ROOM,216,ANG MO KIO AVE 1,216 ANG MO KIO AVE 1 SINGAPORE,1.366272,103.841465,...,7.606769,61.59,76635.81750,1049.805719,92.9,48775.026911,668.151054,79193.094510,1084.836911,22901
3,1990-01,1990,ANG MO KIO,2,3 ROOM,308,ANG MO KIO AVE 1,308 ANG MO KIO AVE 1 SINGAPORE,1.365485,103.844025,...,8.287972,61.59,76635.81750,1143.818172,92.9,48775.026911,727.985476,79193.094510,1181.986486,22901
4,1990-01,1990,ANG MO KIO,2,4 ROOM,211,ANG MO KIO AVE 3,211 ANG MO KIO AVE 3 SINGAPORE,1.369226,103.841652,...,11.484353,61.59,129891.21610,1603.595261,93.3,82315.112540,1016.235957,133650.125891,1650.001555,22901


In [4]:
df.duplicated().sum()   # rectify duplicated values

210

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
len(df)    # data after cleaning

80164

### Data Integration: Enhance HDB data

In [7]:
# Merge datasets based on address and resale price (in event where multiple flats are from sale address)
df = pd.merge(df, locations[['street_name', 'month', 'resale_price', 'block', 'flat_type',
                             'latitude', 'longitude', 'storey',
                                'price_psm', 'price cpi_adj', 'price_psm cpi_adj', 'Core CPI']],
                                  on=['street_name', 'month', 'resale_price', 'block', 'flat_type'], how='left')

In [8]:
len(df)     # data has extra rows after join

81021

In [9]:
df.duplicated(['street_name', 'month', 'resale_price', 'block', 'flat_type']).sum()

1177

In [10]:
df = df.drop_duplicates(['street_name', 'month', 'resale_price', 'block', 'flat_type'])

In [11]:
def extract_year(lease):
    return int(lease.split()[0])

df['remaining_lease_years'] = df['remaining_lease'].apply(extract_year)

In [12]:
df.head()    # dataset with new features like coordinates and inflation accounted prices

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,latitude,longitude,storey,price_psm,price cpi_adj,price_psm cpi_adj,Core CPI,remaining_lease_years
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,1.362001,103.853870,11.0,5272.727273,239761.0657,5449.115130,96.763,61
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,1.370919,103.838206,2.0,3731.343284,258363.2173,3856.167423,96.763,60
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,1.380490,103.835646,2.0,3910.447761,270764.6518,4041.263459,96.763,62
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,1.366376,103.856612,5.0,3897.058824,273865.0104,4027.426624,96.763,62
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,1.381033,103.835134,2.0,3955.223881,273865.0104,4087.537469,96.763,62


In [13]:
len(df)     # data after integrating external data 

79844

### Data Integration: Addition of Malls and MRTs into dataset

In [14]:
df['FLAG'] = 'HDB'    # all existing records in df are HDB

In [15]:
df.head(1)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,latitude,longitude,storey,price_psm,price cpi_adj,price_psm cpi_adj,Core CPI,remaining_lease_years,FLAG
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,1.362001,103.85387,11.0,5272.727273,239761.0657,5449.11513,96.763,61,HDB


In [16]:
malls.head()

,Mall Name,LATITUDE,LONGITUDE
0,100 AM,1.274588,103.843471
1,313@Somerset,1.301014,103.838361
2,Aperia,1.310474,103.864313
3,Balestier Hill Shopping Centre,1.325596,103.842572
4,Bugis Cube,1.298141,103.855635


In [17]:
len(malls)

155

In [18]:
mrts.tail()

,station_name,type,lat,lng
167,Punggol Point,LRT,1.416932,103.906680
168,Samudera,LRT,1.415955,103.902185
169,Nibong,LRT,1.411865,103.900321
170,Sumang,LRT,1.408501,103.898605
171,Soo Teck,LRT,1.405436,103.897287


In [19]:
len(mrts)

172

In [20]:
mrts['MRT Name'] = mrts.station_name+' '+mrts.type

In [21]:
# Rename columns in the 'malls' DataFrame
malls = malls.rename(columns={'Mall Name': 'street_name', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'})

# add flag
malls['FLAG'] = 'Mall'

In [22]:
# Rename MRT columns
mrts = mrts.rename(columns={'MRT Name': 'street_name', 'lat': 'latitude', 'lng': 'longitude', 'type': 'FLAG'})

# Drop station_name column
mrts = mrts.drop('station_name', axis=1)

In [23]:
# Concatenate mrts and malls to df
df = pd.concat([df,
                mrts,
                malls], ignore_index=True)

In [24]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,latitude,longitude,storey,price_psm,price cpi_adj,price_psm cpi_adj,Core CPI,remaining_lease_years,FLAG
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979.0,61 years 04 months,232000.0,1.362001,103.853870,11.0,5272.727273,239761.0657,5449.115130,96.763,61.0,HDB
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978.0,60 years 07 months,250000.0,1.370919,103.838206,2.0,3731.343284,258363.2173,3856.167423,96.763,60.0,HDB
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980.0,62 years 05 months,262000.0,1.380490,103.835646,2.0,3910.447761,270764.6518,4041.263459,96.763,62.0,HDB
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980.0,62 years 01 month,265000.0,1.366376,103.856612,5.0,3897.058824,273865.0104,4027.426624,96.763,62.0,HDB
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980.0,62 years 05 months,265000.0,1.381033,103.835134,2.0,3955.223881,273865.0104,4087.537469,96.763,62.0,HDB


In [25]:
df.tail()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,latitude,longitude,storey,price_psm,price cpi_adj,price_psm cpi_adj,Core CPI,remaining_lease_years,FLAG
80166,NaN,NaN,NaN,NaN,Gek Poh Shopping Centre,NaN,NaN,NaN,NaN,NaN,NaN,1.348742,103.697740,NaN,NaN,NaN,NaN,NaN,NaN,Mall
80167,NaN,NaN,NaN,NaN,Rochester Mall,NaN,NaN,NaN,NaN,NaN,NaN,1.305408,103.788447,NaN,NaN,NaN,NaN,NaN,NaN,Mall
80168,NaN,NaN,NaN,NaN,Taman Jurong Shopping Centre,NaN,NaN,NaN,NaN,NaN,NaN,1.334845,103.720462,NaN,NaN,NaN,NaN,NaN,NaN,Mall
80169,NaN,NaN,NaN,NaN,West Coast Plaza,NaN,NaN,NaN,NaN,NaN,NaN,1.303586,103.766104,NaN,NaN,NaN,NaN,NaN,NaN,Mall
80170,NaN,NaN,NaN,NaN,Queensway Shopping Centre,NaN,NaN,NaN,NaN,NaN,NaN,1.287618,103.803391,NaN,NaN,NaN,NaN,NaN,NaN,Mall


In [26]:
len(df)

80171

In [27]:
df.to_csv('HDB_dataset.csv')